In [1]:
import json
import random
from firebase_admin import initialize_app, credentials, db

In [2]:
cred = credentials.Certificate('creds.json')

In [3]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [4]:
seed = random.randrange(2 ** 32)
seed

2230573050

In [8]:
players = ["Heiko", "Simon", "Niko", "Dave"]
random.shuffle(players)

In [6]:
with open("board.json") as json_file:
    board = json.load(json_file)

In [9]:
db.reference("test").set({
    'seed': seed,
    'players': players,
    'board': board,
})

In [ ]:
db.reference("test/board").set(board)

In [ ]:
', '.join([str(i) for i in sorted(a+b+2 for a in range(6) for b in range(6))])

In [ ]:
events = db.reference("test/events").get()
db.reference("test/events").set(events[:-3])